In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import os
import json
from folium import plugins
import numpy as np

#Bring in Nigerian shapefile
naija_states = os.path.join("/Users/bunmiajala/naija_map_convert.json")

geo_json_data = json.load(open(naija_states))

#Link to our file
file_url = "/Users/bunmiajala/Documents/NIMR/TB/nimr_heatmap_geocoded_data_tb.csv"

#We want Pandas to treat ID or postcode as objects
nimr = pd.read_csv(file_url, sep=',', dtype={'postcode' : object}
                 )
#incase some column names have some extra padding. We remove strips
nimr.columns = nimr.columns.str.strip()

##drop nul
nimr = nimr[pd.notnull(nimr['latitude'])]

#Let's see what our data looks like
nimr.head()

,Unnamed: 0,N4SEX,N5AGE,N1DATARESG,reference,location,Unnamed: 0.1,accuracy,formatted_address,google_place_id,input_string,latitude,longitude,number_of_results,postcode,status,type,year,period
0,0,MALE,9.0,27/02/2006,TqPhPb2y5KWDKNOn,IKEJA,0,APPROXIMATE,"Ikeja, Nigeria",ChIJmTkq-iiSOxAR8KG73UsyqNc,"IKEJA,,Nigeria",6.601838,3.351486,1,6oqwlt,OK,"locality,political",2006,2006-02-27
1,1,FEMALE,34.0,27/02/2006,PEyiyeqnU4mQJ0om,SURULERE,1,APPROXIMATE,"Surulere, Lagos, Nigeria",ChIJh97Iew-MOxARsY8sPcsPZWI,"SURULERE,,Nigeria",6.492632,3.348967,1,vYDJuF,OK,"administrative_area_level_2,political",2006,2006-02-27
2,2,FEMALE,33.0,27/02/2006,2UqjUzRZVCRL2dqv,ALIMOSHO,2,APPROXIMATE,"Alimosho, Nigeria",ChIJ6cPolh-QOxAR_rCHmFNKObg,"ALIMOSHO,,Nigeria",6.574446,3.256987,1,GT29I0,OK,"political,sublocality,sublocality_level_1",2006,2006-02-27
3,3,MALE,64.0,27/02/2006,6C2PHNgy5LXjMJxY,ETI OSA,3,APPROXIMATE,"Eti Osa, Nigeria",ChIJMXoyEtz2OxARIt71U9xyiw8,"ETI OSA,,Nigeria",6.458985,3.601521,1,82aR2V,OK,"administrative_area_level_2,political",2006,2006-02-27
4,4,MALE,6.0,27/02/2006,c0i6lSpuwHeIGKUg,KOSHOFE,4,APPROXIMATE,"Kosofe, Nigeria",ChIJG0inqbaSOxAReqfAiBq_vy8,"KOSHOFE,,Nigeria",6.598781,3.409463,1,u1vP9M,OK,"administrative_area_level_2,political",2006,2006-02-27


In [2]:
#Instead of looking for the co-ordinates we allow our data to determine from our data means
some_map = folium.Map(location=[nimr['latitude'].mean(), 
                    nimr['longitude'].mean()], tiles = 'cartodbpositron',
                      zoom_start=9)

some_map

In [3]:
#Instead of looking for the co-ordinates we allow our data to determine from our data means
some_map = folium.Map(location=[nimr['latitude'].mean(), 
                    nimr['longitude'].mean()], tiles = 'cartodbpositron',
                      zoom_start=9)

#some_map

tb = nimr[['latitude', 'longitude']].values

tb = tb.tolist()
# plot heatmap
some_map.add_child(plugins.HeatMap(tb, radius=15))


some_map

In [4]:
#markout/overlay Nigeria from the map using the shapefile

folium.GeoJson(geo_json_data).add_to(some_map)

some_map

In [ ]:
#We present same using a circle instead of traditional marker
#creating a map that's centered to our data
some_map = folium.Map(location=[nimr['latitude'].mean(), 
                                nimr['longitude'].mean()], 
                                zoom_start=8)

callback = ('function (row) {' 
                'var circle = L.circle(new L.LatLng(row[0], row[1]), {color: "red",  radius: 500});'
                'return circle};')

some_map.add_child(FastMarkerCluster(nimr[['latitude', 'longitude']].values.tolist(), callback=callback))

#folium.GeoJson(geo_json_data).add_to(some_map)
some_map